In [1]:
!pip install -U pip

    100% |████████████████████████████████| 1.5MB 17.2MB/s ta 0:00:01
fastai 1.0.60 requires nvidia-ml-py3, which is not installed.
  Found existing installation: pip 10.0.1
    Uninstalling pip-10.0.1:
      Successfully uninstalled pip-10.0.1
You are using pip version 20.1.1, however version 20.2b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
!pip install transformers

     |████████████████████████████████| 660 kB 11.4 MB/s eta 0:00:01
     |████████████████████████████████| 883 kB 48.9 MB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 67.7 MB/s eta 0:00:01
     |████████████████████████████████| 3.8 MB 69.5 MB/s eta 0:00:01
     |████████████████████████████████| 675 kB 66.4 MB/s eta 0:00:01
     |████████████████████████████████| 298 kB 63.4 MB/s eta 0:00:01
  Created wheel for sacremoses: filename=sacremoses-0.0.43-py3-none-any.whl size=893264 sha256=150e8f234583c2f1a726a22e2ec271b8a85af12d2c5577aa46c77683cea4025b
  Stored in directory: /home/ec2-user/.cache/pip/wheels/49/25/98/cdea9c79b2d9a22ccc59540b1784b67f06b633378e97f58da2
Successfully built sacremoses


In [3]:
import torch
from transformers import BertModel, BertTokenizer

In [4]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [5]:
granola_ids = tokenizer.encode("granola bars")
print("granola_ids", granola_ids)
print("type of granola_ids", type(granola_ids))
print("granola_tokens", tokenizer.convert_ids_to_tokens(granola_ids))

granola_ids [101, 12604, 6030, 6963, 102]
type of granola_ids <class 'list'>
granola_tokens ['[CLS]', 'gran', '##ola', 'bars', '[SEP]']


In [6]:
granola_ids = torch.LongTensor(granola_ids)
print("granola_ids", granola_ids)
print("type of granola_ids", type(granola_ids))

granola_ids tensor([  101, 12604,  6030,  6963,   102])
type of granola_ids <class 'torch.Tensor'>


In [7]:
model = BertModel.from_pretrained("bert-base-uncased", output_hidden_states=True)
device = "cude" if torch.cuda.is_available() else "cpu"

model = model.to(device)
granola_ids = granola_ids.to(device)

model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [8]:
print(granola_ids.size())
granola_ids = granola_ids.unsqueeze(0)
print(granola_ids.size())

print(type(granola_ids))
with torch.no_grad():
    out = model(input_ids=granola_ids)

    
print(type(out))
print(len(out))
hidden_states = out[2]
print(len(hidden_states))

torch.Size([5])
torch.Size([1, 5])
<class 'torch.Tensor'>
<class 'tuple'>
3
13


In [9]:
print(model)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [10]:
sentence_embedding = torch.mean(hidden_states[-1], dim=1).squeeze()
print(sentence_embedding)
print(sentence_embedding.size())

tensor([ 2.7497e-01,  1.8313e-01, -8.8652e-02,  2.1698e-01,  3.1942e-01,
        -1.1412e-01,  7.4039e-02,  3.7655e-01, -4.1821e-01,  9.9971e-02,
        -9.0241e-02, -2.4298e-01,  1.5542e-01,  4.2042e-01, -2.5547e-01,
         2.9753e-01, -2.9643e-01, -2.5810e-02,  8.5306e-02,  1.0182e-01,
         3.0401e-01, -4.4263e-01,  3.1249e-02,  1.4435e-01,  3.0189e-01,
         7.3914e-02, -2.5580e-01,  3.1383e-01, -1.4688e-01, -1.5202e-01,
         7.0786e-02,  4.0447e-01, -1.1769e-01,  3.1848e-01,  2.8021e-02,
        -1.6934e-01,  3.5639e-01, -2.2931e-01, -1.1899e-01, -1.1182e-01,
        -1.6003e-01,  7.9355e-02,  5.1107e-01,  5.2224e-02, -1.5481e-01,
         2.8228e-02, -1.4365e-01, -4.7737e-01, -5.6638e-01, -4.8802e-01,
        -1.1429e-01,  2.8087e-01, -5.7160e-02,  2.3862e-01,  3.5440e-01,
         5.8237e-01,  1.2777e-01,  1.0363e-01,  3.0538e-01,  2.0989e-01,
         1.1693e-01,  2.6346e-01, -1.5832e-01, -1.1380e-01,  1.7190e-02,
        -3.4661e-02,  1.1470e-01,  3.2024e-02, -1.9